In [2]:
# need to fix the asyncio event loop
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Stan example
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Building...

In file included from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun.hpp:110,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/rev/fun.hpp:7,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math.hpp:19,
                 from /opt/conda/lib/python3.9/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /home/jovyan/.cache/httpstan/4.5.0/models/wfh3pgxe/model_wfh3pgxe.cpp:2:
/opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun/generalized_inverse.hpp: In function ‘Eigen::Matrix<typename stan::value_type<T>::type, EigMat::ColsAtCompileTime, EigMat::RowsAtCompileTime> stan::math::generalized_inverse(const EigMat&)’:
/opt/conda/lib/python3.9/site-packages/httpstan/include/stan/math/prim/fun/generalized_inverse.hpp:34:9: warning: typedef ‘us


Building: 16.4s, done.Messages from stanc:
Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 3.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.31 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.12 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.12 seconds.
  Adjust your expectations accordingly!


In [3]:
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,tau,eta.1,...,eta.7,eta.8,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6,theta.7,theta.8
draws,,,,,,,,,,,,,,,,,,,,,
0,-36.110262,1.000000,0.327249,3.0,7.0,0.0,39.633554,13.145834,13.862475,0.073710,...,0.169720,-1.343421,14.167635,11.238470,4.235081,20.028498,3.027229,-2.227330,15.498574,-5.477304
1,-44.258014,0.872767,0.404547,3.0,7.0,0.0,47.814396,7.043367,0.551003,2.120918,...,1.568510,-0.723536,8.212000,7.649858,7.762083,7.936277,6.850538,7.403839,7.907621,6.644697
2,-42.552290,0.996407,0.336762,4.0,15.0,0.0,51.593331,9.319632,1.512233,0.898427,...,-0.600326,-0.024635,10.678263,9.764378,8.525141,9.485779,12.868676,12.056368,8.411800,9.282378
3,-42.352365,0.893377,0.388903,4.0,15.0,0.0,48.231329,-0.295861,3.299899,-0.027857,...,2.937406,0.385555,-0.387787,-0.564169,3.695146,-3.467930,-1.040277,-0.343546,9.397283,0.976433
4,-37.577375,0.173365,0.327249,3.0,7.0,0.0,42.538248,12.550182,5.633321,0.324533,...,0.095530,-1.666240,14.378381,11.710421,8.045411,16.508423,9.574965,9.437941,13.088333,3.163718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-45.013899,0.892852,0.388903,4.0,15.0,0.0,49.799297,3.726645,1.332717,-0.128269,...,1.465650,2.431394,3.555698,6.483250,4.669425,1.664654,2.708238,4.579617,5.679940,6.967004
3996,-38.759711,0.719531,0.327249,3.0,7.0,0.0,45.612775,9.010980,11.910247,1.588584,...,1.375902,-0.112547,27.931413,-7.227985,15.825243,3.225798,9.878761,7.726680,25.398308,7.670517
3997,-40.515721,0.978756,0.404547,3.0,7.0,0.0,43.061005,9.630610,3.593905,0.081038,...,-0.608611,0.781842,9.921851,5.621283,17.745126,11.739614,9.903062,8.034741,7.443318,12.440477
